# Download and extract dataset

As an initial step, it is of essential importance to download the [Public Coin Dataset](https://drive.google.com/file/d/1cF-u9N7miok5-KUiriTdvA62edxIs1I_) and extract its contents. Moreover, we retrieve the parameters from the previously executed training sessions, which will be instrumental in our ongoing work.


In [1]:
!gdown 1cF-u9N7miok5-KUiriTdvA62edxIs1I_ -O public_coin_dataset.zip
!gdown 1RaRbCZHKc8kIeI6KaznPxO_9VM3tE6TP -O model_checkpoints_pretrained.pth
!unzip -qq -o public_coin_dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1cF-u9N7miok5-KUiriTdvA62edxIs1I_
To: /content/public_coin_dataset.zip
100% 40.6M/40.6M [00:00<00:00, 41.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RaRbCZHKc8kIeI6KaznPxO_9VM3tE6TP
To: /content/model_checkpoints_pretrained.pth
100% 62.2M/62.2M [00:02<00:00, 30.1MB/s]


# Importing
Importing all the libraries utilized in the tasks


In [2]:
import cv2
import os
import random
import torch
import sys

import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

from torch import nn
from torch.nn.functional import pad
from torch.utils.data import Dataset, DataLoader

from torchvision.transforms import ToTensor
from torchvision.transforms.functional import hflip, vflip, resize
from torchvision.transforms.transforms import Compose, RandomHorizontalFlip, RandomVerticalFlip

## Model
I decided to employ the [U-net](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/) neural network architecture for the specific segmentation task, known for its effectiveness in image segmentation and feature extraction. \\

This U-Net model consists of a multi-level structure with four levels of blocks. Each block includes a pair of convolutional layers with batch normalization and ReLU activation functions. There is one additional max-pooling layer in the encoding part, while in the decoding part, up-convolutional layers are utilized instead.

In [3]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=3, out_channels=1, pretrained=False)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip


In [4]:
class CoinDataset(Dataset):
  def __init__(self, image_list: list, patch_size: int, transform=None):
    super(Dataset, self).__init__()
    self.transform = transform
    self.patch_size = patch_size
    self.path_images = image_list

  def __len__(self) -> int:
    return len(self.path_images)

  def __getitem__(self, index):
    original_path = os.path.join(self.path_images[index], "original", "coin.jpeg")
    no_background_path = os.path.join(self.path_images[index], "no_bg", "coin.jpeg")

    original_image = Image.open(original_path).convert("RGB")
    no_background_image = Image.open(no_background_path).convert("L")
    no_background_image = np.asarray(no_background_image)
    no_background_image = (no_background_image != 0) * 1.0
    no_background_image = Image.fromarray(no_background_image)

    if self.transform:
      original_image, no_background_image = self.transform((original_image, no_background_image))

    original_image = resize(original_image, (self.patch_size, self.patch_size), antialias=True)
    no_background_image = resize(no_background_image, (self.patch_size, self.patch_size), antialias=True)

    return original_image, no_background_image

### Augmentation
In light of the dataset's lack of images, a deliberate strategy was employed to incorporate augmentation methods. This strategic decision was motivated by the need to mitigate issues related to overfitting and underfitting. \\

To ensure uniform application of the desired alterations to input images and their corresponding ground truths, a sophisticated approach was adopted involving customising Transform modules within the PyTorch library.

In [5]:
class CustomRandomHorizontalFlip(RandomHorizontalFlip):

  def __call__(self, samples):
    image, gt = samples
    if random.random() < self.p:
      image = hflip(image)
      gt = hflip(gt)
    return image, gt

class CustomRandomVerticalFlip(RandomVerticalFlip):

  def __call__(self, samples):
    image, gt = samples
    if random.random() < self.p:
      image = vflip(image)
      gt = vflip(gt)
    return image, gt

class CustomToTensor(ToTensor):

  def __call__(self, samples):
    image, gt = samples
    image = super().__call__(image)
    gt = super().__call__(gt)
    return image, gt

In [6]:
# Reproducibility
seed_value = 12
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)

# Configure
transform = Compose([
    CustomToTensor(),
    CustomRandomHorizontalFlip(),
    CustomRandomVerticalFlip(),
])

# Load paths
root_dir = "public_coin_dataset"
paths = [os.path.join(root_dir, path) for path in os.listdir(root_dir)]
random.shuffle(paths)
split_index = int(len(paths) * 70 / 100)
train_paths = paths[:split_index]
test_paths = paths[split_index:]

training_batch_size = 16
dev_batch_size = 4
training_data = CoinDataset(image_list = train_paths, patch_size = 384, transform=transform)
train_dataloader = DataLoader(training_data, batch_size=training_batch_size, shuffle=True)
test_data = CoinDataset(test_paths, patch_size = 384, transform=CustomToTensor())
test_dataloader = DataLoader(test_data, batch_size=dev_batch_size, shuffle=False)

device = "cpu"
if torch.cuda.is_available():
    model = model.to('cuda')
    torch.cuda.manual_seed(seed_value)
    device = "cuda"

## Training

The dataset was methodically split into two segments, with a distribution ratio of 70% for training data and 30% for testing data. This division was meticulously designed to facilitate effective model training and comprehensive testing. \\

In the context of this project, the choice of loss function fell upon MSE (Mean Squared Error) due to its suitability for the task. Additionally, RMSProp was selected as the optimizer to fine-tune the model's performance."

In [ ]:
# Train
epochs = 30
best_accuracy = 0.0
loss_fn = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

# Load pre-trained if available
if os.path.exists("model_checkpoints.pth"):
  checkpoint = torch.load("model_checkpoints.pth")
  model.load_state_dict(checkpoint["checkpoints"])
  optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
  best_accuracy = checkpoint["best_accuracy"]
  print("Loaded Pre-trained")

for epoch in range(epochs):
  training_loss, dev_loss, accuracy = 0., 0., 0.

  model.train(True)
  for (inputs, targets) in train_dataloader:
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    predictions = model(inputs)
    loss = loss_fn(predictions, targets)

    loss.backward()
    optimizer.step()
    training_loss += loss.item() * inputs.size(0)

  model.eval()
  with torch.no_grad():
    for (inputs, targets) in test_dataloader:
      inputs, targets = inputs.to(device), targets.to(device)
      predictions = model(inputs)
      dev_loss += loss_fn(predictions, targets).item() * inputs.size(0)
      acc = (torch.round(predictions) == targets).float().mean()
      accuracy += acc

  avg_accuracy = accuracy / len(test_dataloader) * 100
  avg_training_loss = training_loss / len(train_dataloader.sampler)
  avg_dev_loss = dev_loss / len(test_dataloader.sampler)

  if best_accuracy < avg_accuracy:
    best_accuracy = avg_accuracy
    torch.save({
        'checkpoints': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_accuracy': best_accuracy
        }, "model_checkpoints.pth")

  print(f"EPOCH [{epoch + 1}/{epochs}] TRAIN LOSS: {avg_training_loss:>7f}, DEV LOSS: {avg_dev_loss:>7f}, Accuracy: {avg_accuracy:.2f}%, Best Accuracy: {best_accuracy:.2f}%")

## Show Results

In [ ]:
# Show Results
dataloader = DataLoader(
    CoinDataset(random.choices(test_paths, k = 8), patch_size=384, transform=CustomToTensor()),
    batch_size=1,
    shuffle=False
)
accuracy = 0.
if os.path.exists("model_checkpoints_pretrained.pth"):
  checkpoint = torch.load("model_checkpoints_pretrained.pth")
  model.load_state_dict(checkpoint["checkpoints"])
model.eval()

with torch.no_grad():
  for i, (inputs, targets) in enumerate(dataloader):
    inputs, targets = inputs.to(device), targets.to(device)
    predictions = model(inputs)

    fig, ax = plt.subplots(nrows=1, ncols=3)
    ax[0].imshow(inputs[0].cpu().swapaxes(0, 2))
    ax[0].set_title("Original")
    ax[1].imshow(predictions[0].cpu().swapaxes(0, 2))
    ax[1].set_title("Predicted")
    ax[2].imshow(targets[0].cpu().swapaxes(0, 2))
    ax[2].set_title("Ground Truth")
    plt.show()

# Hough Transform

An alternative approach was explored during the experimentation phase, which diverged from deep learning methodologies and instead leveraged the OpenCV Hough Transform function for circle detection.

Although this approach resulted in a slightly lower accuracy, it demonstrated a notable advantage in terms of execution speed, making it a pragmatic choice for specific scenarios.

In [ ]:
def apply_hough_transform(image_path: os.path) -> np.ndarray:
  if not os.path.exists(image_path):
    return

  original_path = os.path.join(image_path, "original", "coin.jpeg")
  original_image = cv2.imread(original_path, cv2.IMREAD_COLOR)

  # Elaborate circle
  grayscale_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
  blurred_image = cv2.GaussianBlur(grayscale_image, (5, 5), 0)
  rows = blurred_image.shape[0]
  circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, 1.5, rows / 2)

  # Detect bigger cicle
  big_circle = circles[0][0]
  center = (int(np.floor(big_circle[0])), int(np.floor(big_circle[1])))
  radius = int(np.floor(big_circle[2]))
  result_image = original_image.copy()
  cv2.circle(result_image, center, radius, (255, 255, 255), thickness=cv2.FILLED)

  bin_result = ((result_image == (255, 255, 255)) * (255, 255, 255))[:, :, 0]
  return bin_result


root_dir = "public_coin_dataset"
path_images = [os.path.join(root_dir, path) for path in os.listdir(root_dir)]
random.shuffle(path_images)

accuracy = 0.0
not_detected = 0

for image_path in path_images:
  try:
    no_background_path = os.path.join(image_path, "no_bg", "coin.jpeg")
    no_background = cv2.imread(no_background_path, cv2.IMREAD_GRAYSCALE)

    bin_result = apply_hough_transform(image_path)
    bin_background = (no_background != 0) * 255

    accuracy += (bin_result == bin_background).mean()

  except Exception:
    not_detected += 1
    continue

avg_accuracy = accuracy / len(path_images) * 100
print(f"Accuracy: {avg_accuracy:.2f}%")
print(f"Number of not detected image: {not_detected}")

In [ ]:
image_path = random.choice(path_images)

original_path = os.path.join(image_path, "original", "coin.jpeg")
original_image = cv2.imread(original_path, cv2.IMREAD_COLOR)
no_background_path = os.path.join(image_path, "no_bg", "coin.jpeg")
no_background = cv2.imread(no_background_path, cv2.IMREAD_GRAYSCALE)
bin_result = apply_hough_transform(image_path)
bin_background = (no_background != 0) * 255

fig, ax = plt.subplots(nrows=1, ncols=4)
ax[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original")
ax[0].axis('off')

ax[1].imshow(bin_result, cmap = 'gray')
ax[1].set_title("Result")
ax[1].axis('off')

ax[2].imshow(bin_background, cmap = 'gray')
ax[2].set_title("Ground Truth")
ax[2].axis('off')

ax[3].imshow(bin_result - bin_background, cmap = 'gray')
ax[3].set_title("Error")
ax[3].axis('off')

plt.show()